In [512]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import json
import numpy as np

In [513]:
#retrieving the url for FOMC Statement
html_page= requests.get('https://www.federalreserve.gov/newsevents/pressreleases/monetary20190320a.htm') 
soup= BeautifulSoup(html_page.content, 'html.parser')

In [514]:
#preview the soup... MMM soup
print(soup.prettify()[:1000])

<!DOCTYPE doctype html>
<html class="no-js" lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1.0, minimum-scale=1.0 maximum-scale=1.6, user-scalable=1" name="viewport"/>
  <meta content=" Information received since the Federal Open Market Committee met in January indicates that the labor market remains strong but that growth of economic activity" name="description">
   <meta content="Federal Reserve issues FOMC statement" property="og:title">
    <meta content="Board of Governors of the Federal Reserve System" property="og:site_name">
     <meta content="article" property="og:type"/>
     <meta content=" Information received since the Federal Open Market Committee met in January indicates that the labor market remains strong but that growth of economic activity" property="og:description"/>
     <meta content="" property="og:image"/>
     <meta content="summary" name="twitter:card"/>


In [515]:
#search for the text in the article, isolate and save text
article = soup.find("div",attrs={"class":"col-xs-12 col-sm-8 col-md-8"}).getText()

In [516]:
article

"\nInformation received since the Federal Open Market Committee met in January indicates that the labor market remains strong but that growth of economic activity has slowed from its solid rate in the fourth quarter. Payroll employment was little changed in February, but job gains have been solid, on average, in recent months, and the unemployment rate has remained low. Recent indicators point to slower growth of household spending and business fixed investment in the first quarter. On a 12-month basis, overall inflation has declined, largely as a result of lower energy prices; inflation for items other than food and energy remains near 2 percent. On balance, market-based measures of inflation compensation have remained low in recent months, and survey-based measures of longer-term inflation expectations are little changed.\nConsistent with its statutory mandate, the Committee seeks to foster maximum employment and price stability. In support of these goals, the Committee decided to ma

In [517]:
r = requests.get('https://www.federalreserve.gov/json/ne-press.json')

In [518]:
c = r.content

In [519]:
json_file = BeautifulSoup(c)

In [520]:
json.loads(json_file.text[0:3579] + ']')

[{'d': '3/25/2019 2:00:00 PM',
  't': 'Federal and state financial regulatory agencies issue interagency statement on supervisory practices regarding financial institutions affected by flooding in the Midwest',
  'pt': 'Banking and Consumer Regulatory Policy',
  'l': '/newsevents/pressreleases/bcreg20190325a.htm'},
 {'d': '3/22/2019 5:00:00 PM',
  't': 'Federal Reserve Board announces approval of proposals by First Interstate Bank',
  'pt': 'Orders on Banking Applications',
  'l': '/newsevents/pressreleases/orders20190322a.htm'},
 {'d': '3/22/2019 12:00:00 PM',
  't': 'Federal Reserve System publishes annual financial statements',
  'pt': 'Other Announcements',
  'l': '/newsevents/pressreleases/other20190322a.htm'},
 {'d': '3/21/2019 12:00:00 PM',
  't': 'Federal Reserve Board publishes report containing summary information on debit card transactions in 2017',
  'pt': 'Other Announcements',
  'l': '/newsevents/pressreleases/other20190321a.htm'},
 {'d': '3/21/2019 11:00:00 AM',
  't': '

In [521]:
our_list = json_file.text.split("Federal Reserve issues FOMC statement")

In [522]:
links = []
for element in our_list[1:]:
    links.append('https://www.federalreserve.gov/newsevents' + element.split('newsevents')[1][0:36])
links

['https://www.federalreserve.gov/newsevents/pressreleases/monetary20190320a.htm',
 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20190130a.htm',
 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20181219a.htm',
 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20181108a.htm',
 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20180926a.htm',
 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20180801a.htm',
 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20180613a.htm',
 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20180502a.htm',
 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20180321a.htm',
 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20180131a.htm',
 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20171213a.htm',
 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20171101a.htm',
 'https://www.fe

In [523]:
len('/pressreleases/bcreg20190325a.htm')

33

In [524]:
documents=[]
for link in links:
    html_page= requests.get(link) 
    soup= BeautifulSoup(html_page.content, 'html.parser')
    article = soup.find("div",attrs={"class":"col-xs-12 col-sm-8 col-md-8"}).getText()
    documents.append(article)

In [525]:
listoftags=[]
for doc in documents:
    listoftags.append([doc,'dovish'])

In [526]:
listoftags[0][1]

'dovish'

In [527]:
df= pd.DataFrame(listoftags)

In [528]:
df.to_excel("FOMC.xlsx")

In [529]:
df=pd.read_excel("FOMC copy.xlsx")

In [530]:
df.drop(columns = 'Unnamed: 2', inplace= True)

In [531]:
df.columns = ["documents", "label"]

In [532]:
df.describe()

,documents,label
count,63,63
unique,63,2
top,\n\nPolicy Normalization Principles and Plans\...,dovish
freq,1,53


In [533]:
df.groupby('label').describe()

documents                                                            \
            count unique                                                top   
label                                                                         
dovish         53     53  \n\nPolicy Normalization Principles and Plans\...   
hawkish        10     10  \nInformation received since the Federal Open ...   

              
        freq  
label         
dovish     1  
hawkish    1

In [534]:
def clean_data(data):
    cleaned_data = []
    for line in data:
        if not '[' in line and not ']' in line:
            for symbol in ",.?!''\n":
                line = line.replace(symbol, '').lower()
            cleaned_data.append(line)
    return "".join(cleaned_data)
clean_data(data_without_brackets)

'information received since the federal open market committee met in january indicates that the labor market remains strong but that growth of economic activity has slowed from its solid rate in the fourth quarter payroll employment was little changed in february but job gains have been solid on average in recent months and the unemployment rate has remained low recent indicators point to slower growth of household spending and business fixed investment in the first quarter on a 12-month basis overall inflation has declined largely as a result of lower energy prices; inflation for items other than food and energy remains near 2 percent on balance market-based measures of inflation compensation have remained low in recent months and survey-based measures of longer-term inflation expectations are little changedconsistent with its statutory mandate the committee seeks to foster maximum employment and price stability in support of these goals the committee decided to maintain the target ra

In [535]:
df.documents[1] == df.documents[2] 

False

In [536]:
from nltk.corpus import stopwords

In [537]:
stopwords.words('english')[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [538]:
import string
def text_process(mess):
    # Check characters to see if they are in punctuation
    nopunc = [doc for doc in mess if doc not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [539]:
df.head()

,documents,label
0,\nInformation received since the Federal Open ...,dovish
1,\nInformation received since the Federal Open ...,dovish
2,\nInformation received since the Federal Open ...,hawkish
3,\nInformation received since the Federal Open ...,dovish
4,\nInformation received since the Federal Open ...,hawkish


In [540]:
df['documents'].head(10).apply(text_process)

0    [Information, received, since, Federal, Open, ...
1    [Information, received, since, Federal, Open, ...
2    [Information, received, since, Federal, Open, ...
3    [Information, received, since, Federal, Open, ...
4    [Information, received, since, Federal, Open, ...
5    [Information, received, since, Federal, Open, ...
6    [Information, received, since, Federal, Open, ...
7    [Information, received, since, Federal, Open, ...
8    [Information, received, since, Federal, Open, ...
9    [Information, received, since, Federal, Open, ...
Name: documents, dtype: object

In [541]:
from sklearn.feature_extraction.text import CountVectorizer

In [542]:
df.head()

,documents,label
0,\nInformation received since the Federal Open ...,dovish
1,\nInformation received since the Federal Open ...,dovish
2,\nInformation received since the Federal Open ...,hawkish
3,\nInformation received since the Federal Open ...,dovish
4,\nInformation received since the Federal Open ...,hawkish


In [561]:
from imblearn.over_sampling import SMOTE, ADASYN
print(df['label'].value_counts()) #Previous original class distribution
X_resampled, y_resampled = SMOTE().fit_sample(df['documents'], df['label']) 

dovish     53
hawkish    10
Name: label, dtype: int64


ValueError: could not convert string to float: "\nInformation received since the Federal Open Market Committee met in January indicates that the labor market remains strong but that growth of economic activity has slowed from its solid rate in the fourth quarter. Payroll employment was little changed in February, but job gains have been solid, on average, in recent months, and the unemployment rate has remained low. Recent indicators point to slower growth of household spending and business fixed investment in the first quarter. On a 12-month basis, overall inflation has declined, largely as a result of lower energy prices; inflation for items other than food and energy remains near 2 percent. On balance, market-based measures of inflation compensation have remained low in recent months, and survey-based measures of longer-term inflation expectations are little changed.\nConsistent with its statutory mandate, the Committee seeks to foster maximum employment and price stability. In supp

In [545]:
bow_transformer = CountVectorizer(analyzer=text_process).fit(df['documents'])

print(len(bow_transformer.vocabulary_))
bow_transformer.vocabulary_

1067


{'Information': 126,
 'received': 844,
 'since': 930,
 'Federal': 103,
 'Open': 171,
 'Market': 153,
 'Committee': 74,
 'met': 684,
 'January': 132,
 'indicates': 597,
 'labor': 628,
 'market': 667,
 'remains': 868,
 'strong': 976,
 'growth': 551,
 'economic': 445,
 'activity': 241,
 'slowed': 937,
 'solid': 946,
 'rate': 832,
 'fourth': 531,
 'quarter': 825,
 'Payroll': 176,
 'employment': 460,
 'little': 647,
 'changed': 333,
 'February': 102,
 'job': 620,
 'gains': 537,
 'average': 297,
 'recent': 845,
 'months': 701,
 'unemployment': 1036,
 'remained': 866,
 'low': 653,
 'Recent': 192,
 'indicators': 599,
 'point': 779,
 'slower': 938,
 'household': 567,
 'spending': 952,
 'business': 322,
 'fixed': 519,
 'investment': 617,
 'first': 517,
 '12month': 6,
 'basis': 305,
 'overall': 745,
 'inflation': 600,
 'declined': 401,
 'largely': 632,
 'result': 881,
 'lower': 654,
 'energy': 463,
 'prices': 799,
 'items': 619,
 'food': 524,
 'near': 711,
 '2': 15,
 'percent': 766,
 'balance': 3

In [546]:
documents_bow = bow_transformer.transform(df['documents'])

In [547]:
print('Shape of Sparse Matrix: ', documents_bow.shape)
print('Amount of Non-Zero occurences: ', documents_bow.nnz)

Shape of Sparse Matrix:  (63, 1067)
Amount of Non-Zero occurences:  14290


In [548]:
sparsity = (100.0 * documents_bow.nnz / (documents_bow.shape[0] * documents_bow.shape[1]))
print('sparsity:', round(sparsity))

sparsity: 21


In [549]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(documents_bow)

In [550]:
documents_tfidf = tfidf_transformer.transform(documents_bow)
print(documents_tfidf)

  (0, 1060)	0.041389044804090946
  (0, 1043)	0.04428037501621468
  (0, 1036)	0.03202146992948094
  (0, 1018)	0.052049666442273934
  (0, 1005)	0.09023688897011548
  (0, 1002)	0.03410040147181566
  (0, 1000)	0.1343530958883752
  (0, 999)	0.05628725634353538
  (0, 997)	0.051078674109008855
  (0, 993)	0.09167858704452532
  (0, 976)	0.12522348704066175
  (0, 961)	0.029612612989913974
  (0, 953)	0.029612612989913974
  (0, 952)	0.03055277490603717
  (0, 946)	0.11731506013707327
  (0, 938)	0.09514452405062326
  (0, 937)	0.0671765479441876
  (0, 933)	0.04208584431735919
  (0, 930)	0.030078962990038493
  (0, 908)	0.030078962990038493
  (0, 881)	0.1322421090047724
  (0, 868)	0.06505536153766353
  (0, 866)	0.07878386451269744
  (0, 863)	0.054092725644679894
  (0, 845)	0.06505536153766353
  :	:
  (62, 119)	0.030148427835373703
  (62, 115)	0.027850521075420187
  (62, 113)	0.09975961678277745
  (62, 112)	0.05194919708927474
  (62, 105)	0.04488093618889678
  (62, 103)	0.024569695428493136
  (62, 100)	

In [551]:
!pip install imblearn

You are using pip version 19.0.2, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [552]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB().fit(documents_tfidf, df['label'])

In [553]:
all_predictions = model.predict(documents_tfidf)
print(all_predictions)

['dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish'
 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish'
 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish'
 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish'
 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish'
 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish'
 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish'
 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish' 'dovish']


In [554]:
from sklearn.metrics import classification_report
print(classification_report(df['label'], all_predictions))

              precision    recall  f1-score   support

      dovish       0.84      1.00      0.91        53
     hawkish       0.00      0.00      0.00        10

   micro avg       0.84      0.84      0.84        63
   macro avg       0.42      0.50      0.46        63
weighted avg       0.71      0.84      0.77        63



/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [555]:
from sklearn.model_selection import train_test_split

msg_train, msg_test, label_train, label_test = train_test_split(df['documents'], df['label'], test_size=0.2)

In [556]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)),
    ('tfidf', TfidfTransformer()),
    ('classifier', MultinomialNB()),
])

In [557]:
# k_fold = KFold(n_splits = 10)
# nb_f1_scores = []
# nb_conf_mat = np.array([[0, 0], [0, 0]])

# for train_indices, test_indices in k_fold.split(data):

#     train_text = data.iloc[train_indices]['sentence'].values
#     train_y = data.iloc[train_indices]['isRelevant'].values

#     test_text = data.iloc[test_indices]['sentence'].values
#     test_y = data.iloc[test_indices]['isRelevant'].values

#     vectorized_text = nb_pipeline.fit_transform(train_text)

#     sm = SMOTE(ratio = 1.0)
#     train_text_res, train_y_res = sm.fit_sample(vectorized_text, train_y)

#     clf = MultinomialNB()
#     clf.fit(train_text_res, train_y_res)
#     predictions = clf.predict(nb_pipeline.transform(test_text))

In [558]:
pipeline.fit(msg_train,label_train)

Pipeline(memory=None,
     steps=[('bow', CountVectorizer(analyzer=<function text_process at 0x1a2d697620>,
        binary=False, decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        max_features=None, min_df=1, ngram_range=(1, 1), preprocessor=None...f=False, use_idf=True)), ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [559]:
predictions = pipeline.predict(msg_test)

In [560]:
print(classification_report(predictions,label_test))

              precision    recall  f1-score   support

      dovish       1.00      0.77      0.87        13
     hawkish       0.00      0.00      0.00         0

   micro avg       0.77      0.77      0.77        13
   macro avg       0.50      0.38      0.43        13
weighted avg       1.00      0.77      0.87        13



/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
